# Get train and validation datasets to local test
Из-за ограничения в 2 загрузки в день решения на kaggle сначала гоняю алгоритмы на собственных тестах


In [35]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer


source_dataset_path = './train/train_dataset.csv'
train_path = './fit/train.csv'
validation_path = './fit/validation.csv'

train_df = pd.read_csv(source_dataset_path, sep=',', index_col=False)

Получаем выборки: тусуем, делим в соотношении 90/10 - данных слишком мало

In [44]:
ratio = 0.9

def get_train_and_validation_datasets(train_df):
    # Нормализация числовых полей
    train_df['StateHolidayDays'] = sklearn.preprocessing.normalize(train_df['StateHolidayDays'].values.reshape(-1, 1), axis=0)
    train_df['SchoolHolidayDays'] = sklearn.preprocessing.normalize(train_df['SchoolHolidayDays'].values.reshape(-1, 1), axis=0)
    train_df['Sales'] = sklearn.preprocessing.normalize(train_df['Sales'].values.reshape(-1, 1), axis=0)
    train_df['PromoDays'] = sklearn.preprocessing.normalize(train_df['PromoDays'].values.reshape(-1, 1), axis=0)
    train_df['Promo2DaysToNext'] = sklearn.preprocessing.normalize(train_df['Promo2DaysToNext'].values.reshape(-1, 1), axis=0)
    train_df['Promo2AllDays'] = sklearn.preprocessing.normalize(train_df['Promo2AllDays'].values.reshape(-1, 1), axis=0)
    train_df['Promo2Days'] = sklearn.preprocessing.normalize(train_df['Promo2Days'].values.reshape(-1, 1), axis=0)
    train_df['MeanByDay'] = sklearn.preprocessing.normalize(train_df['MeanByDay'].values.reshape(-1, 1), axis=0)
    train_df['Mean'] = sklearn.preprocessing.normalize(train_df['Mean'].values.reshape(-1, 1), axis=0)
    train_df['DayOfWeek'] = sklearn.preprocessing.normalize(train_df['DayOfWeek'].values.reshape(-1, 1), axis=0)
    train_df['DayOfMonth'] = sklearn.preprocessing.normalize(train_df['DayOfMonth'].values.reshape(-1, 1), axis=0)
    train_df['Customers'] = sklearn.preprocessing.normalize(train_df['Customers'].values.reshape(-1, 1), axis=0)
    train_df['CompetitionDistance'] = sklearn.preprocessing.normalize(train_df['CompetitionDistance'].values.reshape(-1, 1), axis=0)
    train_df['CompetitionDays'] = sklearn.preprocessing.normalize(train_df['CompetitionDays'].values.reshape(-1, 1), axis=0)
    # Работа с категориальными признаками
    objects = train_df.select_dtypes(include=[object])
    label_encoder = preprocessing.LabelEncoder()
    transf = objects.apply(label_encoder.fit_transform)
    enc = preprocessing.OneHotEncoder(categories='auto')
    enc.fit(transf)
    labels = enc.transform(transf).toarray()
    # Заменяем/добавляем столбцы
    train_df.drop(columns=['Assortment', 'StoreType', 'StateHoliday'])
    for i in range(0, labels.shape[1]):
        train_df[f'x{i}'] = pd.Series(labels[:,i])
    # Перемешиваем
    indexes = np.arange(0, train_df.shape[0])
    np.random.shuffle(indexes)
    pos = int(train_df.shape[0] * ratio)
    train_indexes = indexes[:pos]
    validation_indexes = indexes[pos:]
    train = train_df.loc[train_indexes]
    validation = train_df.loc[validation_indexes]
    return train, validation

train, validation = get_train_and_validation_datasets(train_df)
train.to_csv(train_path, sep=',', index=False)
validation.to_csv(validation_path, sep=',', index=False)

       Unnamed: 0 Assortment  CompetitionDays  CompetitionDistance  Customers  \
0               0          a         0.003640             0.000711   0.002854   
1               1          a         0.003642             0.000711   0.003120   
2               2          a         0.003644             0.000711   0.003321   
3               3          a         0.003645             0.000711   0.003499   
4               4          a         0.003594             0.000711   0.003377   
5               5          a         0.003596             0.000711   0.003349   
6               6          a         0.003597             0.000711   0.003247   
7               7          a         0.003599             0.000711   0.003181   
8               8          a         0.003601             0.000711   0.003326   
9               9          a         0.003603             0.000711   0.003485   
10             10          a         0.003607             0.000711   0.002849   
11             11          a

# Metrics
Реализация собственного RMSPE для локальных тестов

In [21]:
def metrics(source, predict):
    count = np.min([source.shape[0], predict.shape[0]])
    mse = [(source[x] - predict[x]) ** 2 for x in range(0, count)]
    mse = np.sum(mse) / count
    return np.sqrt(mse)

# Fit
Обучение, применяем OneHotEncoding